In [ ]:
!pip install pyspark # pyspark is installed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=885e28bd6684e0906e3e21a21cef273b6da5612de0199998fc645fe64b18d842
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
import sys
from pyspark import SparkContext, SparkConf #importing packages.

In [ ]:
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('CTF_query of task-2')  #spark session is intiated and activated.
sc = SparkContext(conf=conf)

In [ ]:
file2="/content/CTF_index_matrix"
fileRDD = sc.textFile(file2)   #parallelizing the file into rdd.

In [ ]:
def conversion_to_rdd(x): #this process is used to convert back to rdd.
  y = x.split('@') #as we have every line wrod@doc-id1#freq1..... splitting on @ to get word in first element of list and remaining in second part.
  z = y[1].split('+') # Again splitting on + as to represent 2 different documents.
  document_dictionary ={} #initialinzing dictionary for storing doc-id as key and freq as value.
  for t in z :
    n = t.split('#')
    document_dictionary[n[0]] = float(n[1]) #making freq to floats for further processing 

  return (y[0],document_dictionary) #returning a tuple in which first element is word and second element is dictionary of docids as keys and freq as values.

In [ ]:
CTFRDD = fileRDD.map(lambda x : conversion_to_rdd(x)) # rdd has every element of tuples in the mentioned function.

In [ ]:
#CTFRDD.take(1)

In [ ]:
Query = input('Enter your query for CTF :- ') #query is asked from user.

Enter your query for CTF :- JERUSALEM loves MUSLIM regarding nazi in www.goole.com


In [ ]:
from tkinter.constants import X
import re
import string #necessary packages are imported.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english')) #stopwords list is created for removal .
def preprocessing(x):
   x = x.lower()  #query words are lowered 
   array1 = x.split(" ") #splitted based on empty space.
   y = ""
   for x1 in array1: 
     if x1 in stop_words: #stopwords are removed if they are in the list of stopwords.
       continue
     else:
       y = y+ " "+ x1 + " "

   urls = re.compile(r'https?://\S+|www\.\S+')
   text_without_urls = urls.sub('', y) #query urls are removed

   text_without_punctuations = re.sub(f"[{re.escape(string.punctuation)}]", " ", text_without_urls) #https://www.geeksforgeeks.org/python-remove-punctuation-from-string/ #query punctuations are removed.

   useful_words = set()
   array2 = text_without_punctuations.split(" ")
   for x1 in array2:
     if len(x1)>0:  #I used set to add filtered words.
       useful_words.add(x1)

   return useful_words #that set is returned.


  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
words  = preprocessing(Query) #query is preprocessed.

In [ ]:
#print(words)

In [ ]:
CTFRDD = CTFRDD.filter(lambda x : x[0] in words) #filtering our rdd only when words are in the set.

In [ ]:
#CTFRDD.take(1)

In [ ]:
import math
def construction_of_IDF_and_TFIDF(x):
  no_of_documents_with_one_word = len(x[1]) #length of dictionary to calculate IDF which gives you no of documents the word is present.
  top =10000/(1+ no_of_documents_with_one_word) # here idf is calculated for every word and 10000 is standard total documents which is given in the question.
  IDF = math.log(top,10) 

  output = []
  for y in x[1]:
    TF_IDF_of_particular_document = x[1][y] * IDF
    per_document = (y,IDF,TF_IDF_of_particular_document) #for every value TFIDF is calculated  and overridden in dictionary.
    output.append(per_document)
  return tuple(output) #returning as tuple of list of tuples in the form (doc-id,IDF,TFIDF OF every document)



In [ ]:
CTFRDD = CTFRDD.map(lambda x : construction_of_IDF_and_TFIDF(x)) #rdd will be of the tuple of tuple form (((doc-id,IDF,TFIDF),(doc-id2,IDF2)...))

In [ ]:
#CTFRDD.take(4)

In [ ]:
def conversion_to_dictionary(x):
  output ={} #we get every line as (((doc-id,IDF,TFIDF),(doc-id2,IDF2)...))
  for y in x :
    values=(y[1],y[2])
    output[y[0]] = values
  return output #we wrill return dictionary in which elements will be of the form {doc-id:(IDF,TFIDF)}


In [ ]:
#CTFRDD.take(1)
CTFRDD = CTFRDD.map(lambda x : conversion_to_dictionary(x)) #rdd is mapped based on the return values of convert_to_dictionary function.


In [ ]:
#CTFRDD.take(1)
CTFRDD = CTFRDD.flatMap(lambda x: [(doc_id, x[doc_id]) for doc_id  in x]) #As we have every element of the form tuple of the form (word,{doc-id1:(IDF,TFIDF)}...) But our motive is to get documents so iterating in lambda function and taking that key,value pair as tuple and flattening the rdd.
CTFRDD = CTFRDD.groupByKey() ## for same doc-id as key we group all tuples of the form (IDF,TFIDF) 
CTFRDD = CTFRDD.mapValues(list) # we make the iterator of every key to be list.

In [ ]:
#CTFRDD.take(1)

In [ ]:
#DOT product part

In [ ]:
def Dot_product(x):
  dot_product = 0
  for y in x[1]:
    ct   = y[0]
    ctfd = y[1]
    dot_product += ct*ctfd #performing dot product and adding all the vectors.
  return (x[0],dot_product)


In [ ]:
CTFRDD = CTFRDD.map(lambda x : Dot_product(x)) #we get rdd of elements of type (doc-id, dot producted value) after calling the function.

In [ ]:
OutputRDD = CTFRDD.sortBy(lambda x: x[1], ascending=False) #from pyspark documentation ##from pyspark documentation #using lambda function based on values sorted in descending order.
OutputRDD = OutputRDD.zipWithIndex() # To get first 10 elements of RDD without getting out of rdd used zipindex which gives indexing for every elements.
OutputRDD = OutputRDD.filter(lambda x : x[1]<=9) # taking first 10 document ids based on ids.
OutputRDD = OutputRDD.map(lambda x : (x[0][0],x[0][1])) #taking as tuples.


In [ ]:
#OutputRDD.take(2)

In [ ]:
import os
folder_path = "/content/qID"
OutputRDD.saveAsTextFile(folder_path) #writing back to specifed folder.